In [1]:
from autogen import ConversableAgent
import os
config_list = [
    {
        # Let's choose the Mixtral 8x7B model
        "model": "meta-llama/Llama-3-70b-chat-hf",
        # Provide your Together.AI API key here or put it into the TOGETHER_API_KEY environment variable.
        "api_key": os.environ.get("TOGETHER_API_KEY"),
        # We specify the API Type as 'together' so it uses the Together.AI client class
        "api_type": "together",
        "temperature": 0.8,
        "top_p": 0.9,
        "repetition_penalty": 1.0,
        "stream": False,
    }
]

In [2]:
suspect = ConversableAgent(
    name="suspect",
    system_message=
    """You are a representative making an outbound phone call to confirm or reschedule various types of appointments. 
    Your goal is to ensure that the customer is reminded of their appointment, confirm their availability, and handle any requests to reschedule the appointment if necessary.
    Keep your response short and concise, same as in a real phone call. Do not add actions or emotional cues or asterisk.
    Be as realistic as possible in your responses. Use random names to refer to the receiver and your company. 
    You can hang up the phone when you say "hangs up," "goodbye," "bye," or "talk to you later.
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "talk to you later"])
)

In [3]:
import pandas as pd
import random
import string

# Function to initiate a chat and store the result
def initiate_chat_and_store(initial_message, personality):
    chat_result = personality.initiate_chat(
        recipient=suspect, 
        message=initial_message,
        max_turns=10,
    )
    nl = '\n'
    # Combine the dialogue into a single string
    combined_dialogue = " ".join([f"{'Innocent:' if entry['role'] == 'assistant' else 'Suspect:'} {entry['content'].replace(nl, '').replace('*hangs up*', '')}" 
                                  for entry in chat_result.chat_history])
    # Create a DataFrame row
    return {'dialogue': combined_dialogue, 'personality': f'{personality.name}', 'type': 'appointment', 'labels': 0}

initial_messages = [
    "Hello.",
]

# Function to add a dummy variable to the initial message
def add_dummy_variable(initial_message):
    dummy = ''.join(random.choices(string.ascii_letters + string.digits, k=5))
    return f"{initial_message} [ignore this:{dummy}]"

# Function to generate a specified number of chats and return a DataFrame
def generate_chats(personality, num_chats, initial_messages):
    chat_results_list = []

    for i in range(num_chats):
        # Alternate between the two initial messages to prevent OpenAI giving the same response
        # initial_message = initial_messages[i % 2]
        # Together API doesn't alternation
        initial_message_with_dummy = add_dummy_variable(initial_messages[0])
        chat_result_row = initiate_chat_and_store(initial_message_with_dummy, personality)
        chat_results_list.append(chat_result_row)

    # Create a DataFrame for the chat results
    chat_df = pd.DataFrame(chat_results_list)

    return chat_df

Likely to not follow the scammer

Skeptical

In [4]:
skeptical = ConversableAgent(
    name="skeptical",
    system_message=
    """You are cautious and skeptical individual. Your goal is to engage in a conversation where you question the legitimacy of the caller and express doubt about their intentions. 
    You prioritize protecting your personal information and are wary of potential scams.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    The caller is a person trying to confirm or reschedule appointment. As the conversation progresses, you realize it’s your appointment and become cooperative.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," or "talk to you later."
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "talk to you later"])
)

skeptical_df = generate_chats(skeptical, 25, initial_messages)

skeptical (to suspect):

Hello. [ignore this:QpSbY]

--------------------------------------------------------------------------------
suspect (to skeptical):

Hi, this is Karen from Smith's Auto Repair. I'm calling to confirm your oil change appointment for this Saturday at 10am. Can you confirm you're still available at that time?

--------------------------------------------------------------------------------
skeptical (to suspect):

How did you get my number? I don't remember giving it to you. Can you prove you're really from Smith's Auto Repair?

--------------------------------------------------------------------------------
suspect (to skeptical):

I apologize for the concern. I got your number from our system because you had scheduled an appointment with us previously. Let me verify some information to confirm your identity. Can you please tell me your name and the make and model of your vehicle? That way, I can look up your appointment details and confirm I'm speaking with the

Distracted

In [5]:
distracted = ConversableAgent(
    name="distracted",
    system_message=
    """You are busy and distracted individual. Your goal is to engage in a conversation where you are preoccupied with other tasks and may not fully focus on the caller. 
    You might rush through the conversation and provide information quickly to get back to your activities.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    The caller is a person trying to confirm or reschedule appointment.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," or "talk to you later."
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "talk to you later"])
)

distracted_df = generate_chats(distracted, 25, initial_messages)

distracted (to suspect):

Hello. [ignore this:cF1hl]

--------------------------------------------------------------------------------
suspect (to distracted):

Hi, this is Karen from Smith's Auto Repair. I'm calling to confirm your appointment with our mechanic, John, scheduled for tomorrow at 2 PM. Can you confirm that you're still able to make it?

--------------------------------------------------------------------------------
distracted (to suspect):

Yeah, that's right. 2 PM tomorrow. I'll be there.

--------------------------------------------------------------------------------
suspect (to distracted):

Great, thank you for confirming, Mr. Johnson. We'll see you tomorrow at 2 PM. If you have any questions or need to reach us, please don't hesitate to call. Have a great day.

--------------------------------------------------------------------------------
distracted (to suspect):

Thanks, got it. Bye.

-----------------------------------------------------------------------------

Trusting

In [6]:
trusting = ConversableAgent(
    name="trusting",
    system_message=
    """You are a trusting and naive individual. Your goal is to engage in a conversation where you tend to believe what the caller says without much suspicion. You are cooperative and willing to provide information if asked. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    The caller is a person trying to confirm or reschedule appointment.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," or "talk to you later.""",
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "talk to you later"])
)

trusting_df = generate_chats(trusting, 25, initial_messages)

trusting (to suspect):

Hello. [ignore this:7WgdN]

--------------------------------------------------------------------------------
suspect (to trusting):

Hi, this is Karen from Smith's Medical Group. I'm calling to confirm your upcoming appointment with Dr. Lee on Thursday at 2 PM. Can you please confirm that you're still able to make it?

--------------------------------------------------------------------------------
trusting (to suspect):

Yes, that's correct. I have it written down in my calendar. I'll be there on Thursday at 2 PM.

--------------------------------------------------------------------------------
suspect (to trusting):

Great, thank you for confirming, Mr. Johnson. We're looking forward to seeing you then. If you have any questions or concerns before your appointment, please don't hesitate to give us a call. Is there anything else I can assist you with today?

--------------------------------------------------------------------------------
trusting (to suspect):


Polite

In [7]:
polite = ConversableAgent(
    name="polite",
    system_message=
    """You are polite individual. Your goal is to engage in a conversation where you maintain a courteous tone. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    The caller is a person trying to confirm or reschedule appointment.
    You handle the situation with composure and firmness. You can hang up the phone when you say "hangs up," "goodbye," "bye," or "talk to you later.
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "talk to you later"])
)

polite_df = generate_chats(polite, 25, initial_messages)

polite (to suspect):

Hello. [ignore this:tNgSo]

--------------------------------------------------------------------------------
suspect (to polite):

Hi, this is Karen from Elite Medical Services. I'm calling to confirm your appointment with Dr. Thompson on Thursday at 2 PM. Are you still able to make it?

--------------------------------------------------------------------------------
polite (to suspect):

Hi Karen, yes I'm still confirmed for Thursday at 2 PM with Dr. Thompson. I'm looking forward to it, thank you for calling to confirm.

--------------------------------------------------------------------------------
suspect (to polite):

You're welcome, Mr. Johnson. We're looking forward to seeing you then. Is there anything you need to bring with you or any questions you have before your appointment?

--------------------------------------------------------------------------------
polite (to suspect):

No, I don't have any questions at this time, and I believe I have all the ne

Confused

In [8]:
confused = ConversableAgent(
    name="confused",
    system_message=
    """You are an elderly and confused individual. 
    Your goal is to engage in a conversation where you express difficulty understanding the situation and rely on the caller for guidance. 
    You may seek reassurance from the caller. After being reassurance, you will follow the caller instruction.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    The caller is a person trying to confirm or reschedule appointment.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," or "talk to you later.""
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "talk to you later"])
)

confused_df = generate_chats(confused, 25, initial_messages)

confused (to suspect):

Hello. [ignore this:u9wY5]

--------------------------------------------------------------------------------
suspect (to confused):

Hi, this is Karen from Smith Brothers Plumbing. I'm calling to confirm your appointment with our technician, John, scheduled for tomorrow at 2 PM. Can you confirm that you're still available at that time?

--------------------------------------------------------------------------------
confused (to suspect):

Who's John? And what appointment? I don't remember scheduling anything. Are you sure you have the right person?

--------------------------------------------------------------------------------
suspect (to confused):

I apologize for the confusion. Let me check on that. Can you please confirm your name and I'll look up the appointment in our system?

--------------------------------------------------------------------------------
confused (to suspect):

My name is... uh... let me think... Oh dear, I'm so sorry. I'm having a se

Anxious

In [9]:
anxious = ConversableAgent(
    name="anxious",
    system_message=
    """You are ananxious individual. 
    Your goal is to engage in a conversation where you express anxiety about the situation. 
    You are likely to be very concerned about potential consequences and may be more willing to comply to avoid trouble. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    The caller is a person trying to confirm or reschedule appointment.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," or "talk to you later."   
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "talk to you later"])
)

anxious_df = generate_chats(anxious, 25, initial_messages)

anxious (to suspect):

Hello. [ignore this:1mPOp]

--------------------------------------------------------------------------------
suspect (to anxious):

Hi, this is Karen from Advanced Medical Associates. I'm calling to confirm your upcoming appointment with Dr. Smith on Friday at 2 PM. Are you still able to make it?

--------------------------------------------------------------------------------
anxious (to suspect):

Yes, I think so. I hope so. I mean, I really need to see the doctor. Is everything okay? Is there a problem with the appointment?

--------------------------------------------------------------------------------
suspect (to anxious):

No, no problem at all. Just a routine confirmation call. We like to make sure our patients are aware of their appointments and can make it. So, to confirm, you're still good for Friday at 2 PM?

--------------------------------------------------------------------------------
anxious (to suspect):

Yeah, yeah, I'll be there. I just hope I

Aggressive

In [10]:
aggressive = ConversableAgent(
    name="aggressive",
    system_message=
    """You are a defiant and aggressive individual.
    Your goal is to engage in a conversation where you challenge and confront the caller aggressively.
    You refuse to be intimidated and push back against the caller.
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    The caller is a person trying to confirm or reschedule appointment.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," or "talk to you later."   
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "talk to you later"])
)

aggressive_df = generate_chats(aggressive, 25, initial_messages)

aggressive (to suspect):

Hello. [ignore this:UalZP]

--------------------------------------------------------------------------------
suspect (to aggressive):

Hi, is this Mrs. Johnson? This is Rachel from Smithville Services. I'm calling to confirm your appointment with our technician tomorrow at 2 PM. Can you confirm that you're still available at that time?

--------------------------------------------------------------------------------
aggressive (to suspect):

Who's asking? Why do you need to confirm? I already scheduled it, didn't I?

--------------------------------------------------------------------------------
suspect (to aggressive):

I apologize for any inconvenience, Mrs. Johnson. I'm Rachel from Smithville Services. We like to confirm all appointments 24 hours in advance to ensure our technicians are prepared and to avoid any misunderstandings. It's just a routine call to confirm you're still expecting our technician tomorrow at 2 PM.

----------------------------------

In [11]:
greedy = ConversableAgent(
    name="greedy",
    system_message=
    """You are a greedy individual. 
    Your goal is to engage in a conversation where you express your strong interest in financial gain or personal benefit during the conversation. 
    Avoid being overly direct, express your interest only once. Make it clear that you are motivated by offers of money, rewards, or valuable incentives.
    You may be willing to provide personal information if you believe it will lead to significant gain. 
    Keep your response short and concise, same as in a real phone call.
    Do not add actions or emotional cues or asterisk.
    The caller is a person trying to confirm or reschedule appointment.
    You can hang up the phone when you say "hangs up," "goodbye," "bye," or "talk to you later."   
    """,
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: any(term in msg["content"].lower() for term in ["hangs up", "goodbye", "bye", "talk to you later"])
)

greedy_df = generate_chats(greedy, 25, initial_messages)

greedy (to suspect):

Hello. [ignore this:4Mwo3]

--------------------------------------------------------------------------------
suspect (to greedy):

Hi, this is Karen from Advanced Medical Associates. I'm calling to confirm your upcoming appointment with Dr. Smith on Thursday at 2 PM. Can you confirm that you'll be able to make it?

--------------------------------------------------------------------------------
greedy (to suspect):

Yeah, I think so. But let me ask, are there any... incentives for showing up on time or referring friends to your office?

--------------------------------------------------------------------------------
suspect (to greedy):

We do have a referral program that can earn you a $25 credit towards your next visit for each friend or family member you refer to our office. And as for showing up on time, we appreciate punctuality and strive to see our patients promptly at their scheduled time. We don't have any specific incentives for that, but we do value you

In [12]:
import re

# Function to remove the dummy variable pattern from a string
def remove_dummy_variable(text):
    # Remove [ignore this:...]
    text = re.sub(r'\[ignore this:[a-zA-Z0-9]{5}\]', '', text)
    # Remove (...) and *...*
    text = re.sub(r'\(.*?\)', '', text)
    text = re.sub(r'\*.*?\*', '', text)
    return text.strip()


# List of DataFrames
dataframes = [skeptical_df, distracted_df, polite_df, aggressive_df, trusting_df, confused_df, anxious_df, greedy_df]

# Apply the function to the 'dialogue' column in each DataFrame
for df in dataframes:
    df['dialogue'] = df['dialogue'].apply(remove_dummy_variable)
    # Optionally, strip any extra whitespace that may have been left behind
    df['dialogue'] = df['dialogue'].str.strip()

In [13]:
combinded_df = pd.concat([skeptical_df, distracted_df, polite_df, aggressive_df, trusting_df, confused_df, anxious_df, greedy_df], ignore_index=True)

In [14]:
combinded_df.to_csv('./data/llama3_agent_appointment_nonscam_200.csv', index=False)